In [ ]:
import re
import pandas as pd
from getpass import getpass
import pytds
from pytds.login import NtlmAuth
from pathlib import Path
from collections import defaultdict
import time
password = ''

In [ ]:
host=''
myuser = ''
dump_queries =True

In [ ]:
if password == '':
    password = getpass('please enter your password')

In [ ]:
def run_query(query, database=None):
    global password
    with pytds.connect(
        server=host,
        database=database,
        auth=NtlmAuth(user_name=myuser,password=password),
        as_dict=True,
    ) as conn:
        with conn.cursor() as cur: 
            cur.execute(query)

            c = True
            while c is not None:
                try:
                    return list(cur.fetchall())
                except Exception as e:
                    c = cur.nextset()

# below is the logs found on the server

In [ ]:
pd.DataFrame(run_query("EXEC sys.sp_enumerrorlogs;")).sort_values(by="Archive #")

## please select log

In [ ]:
log = 3

# below is failed password finding.

In [ ]:
failed = run_query(f"EXEC xp_ReadErrorLog {log},1, N'Login fa'")
len(failed)

In [ ]:
ips =defaultdict(list)

In [ ]:
for item in pd.DataFrame(failed)["Text"]:
    m = re.search("CLIENT: (.*)]",item)
    if m is not None:
        ips[m.groups()[0]].append(item)

In [ ]:
{ip:len(ips[ip]) for ip in ips}

In [ ]:
for ip in ips:
    if any(["Login failed for user" in a for a in ips[ip]]):
        print(ip)
        for item in ips[ip]:
            if "Login failed for user" in item:
                print("\t",item)
        print("")

# below is error finding


In [ ]:
system_error_codes =run_query("""
SELECT message_id AS Error, severity AS Severity,  
[Event Logged] = CASE is_event_logged WHEN 0 THEN 'No' ELSE 'Yes' END,
text AS [Description]
FROM sys.messages
WHERE language_id = 1033 /* replace 1040 with the desired language ID, such as 1033 for US English*/
ORDER BY message_id""")
system_error_codes = pd.DataFrame(system_error_codes)

In [ ]:
errors = run_query("EXEC xp_ReadErrorLog 0,1, N'Error', N'Severity'")
len(errors)

present_error_codes = [a.groups()[0] for a in filter(None,[re.match("Error: (.*), Sev",e['Text']) for e in errors])]
pd.DataFrame(present_error_codes)[0].value_counts()

In [ ]:
present_error_codes = list(set(present_error_codes))
for item in present_error_codes:
    print(item,':',system_error_codes.loc[system_error_codes['Error'] == int(item)]['Description'].values[0])
    print()

In [ ]:
for row  in errors:
    if '17836' in row['Text']:
        print(row)